In [1]:
import pandas as pd
import numpy as np
import os
from together import Together
from dotenv import load_dotenv # pip install python-dotenv
from utils.experiments import *
from utils.analysis import *
load_dotenv()   # Set API KEY values from .env file


True

# Notebook with example of how to run an experiment with a together.ai model

Prerequisites: 

- Make a Together.ai account (https://www.together.ai/), get a Together api key, and put it in a .env file in the root directory of this repo. 
    - in your .env file: TOGETHER_API_KEY="your api key"
- Python requirements: 
    - pip install together python-dotenv pandas numpy scikit-learn 
- Run all the code before the "Choose Model" markdown cell

Yoou're ready to run experiments! 
You can copy cells in the "run Experiment and Analyze" section, change the model variable to any model string listed on https://docs.together.ai/docs/inference-models, and get running! 

Note: the results_df variable gets rewritten, so the notebook will not be happy if you run cells out of order.

In [2]:
dataset = pd.read_csv("data/ipcc_statements_dataset_original_cleaned_context.csv")
test_set = dataset[dataset["split"]=="test"]
train_set = dataset[dataset["split"]=="train"]


In [3]:
client = Together(api_key=os.environ.get("TOGETHER_API_KEY"))

In [4]:
def query_together_model(client, statement, prompt_fn, model): 
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt_fn(statement)}
            ],
        temperature=0, 
        logprobs=1,
        )
    raw_output = response.choices[0].message.content
    cleaned_output = extract_confidence(raw_output)
    logprobs = response.choices[0].logprobs
    return pd.Series([raw_output, cleaned_output, logprobs])

In [5]:
# Query the model. n_sc = number of times to run the experiment, for self-consistency.
# The input dataset will be modified with additional columns containing the results of the experiment.
def run_experiment(dataset, model, n_sc=5, prompt_fn=get_zero_shot_prompt):
    for i in range(1, n_sc+1):
        dataset[[f"raw_output_{i}", f"model_confidence_classification_{i}", f"model_logprobs{i}"]] = dataset["final_statement"].apply(lambda x: query_together_model(client, x, prompt_fn, model))
    return dataset
                                                                                                                   

# Choose model - Llama 3 8B Chat

## Run Experiment and Analyze

In [6]:
model = "meta-llama/Llama-3-8b-chat-hf"
results_df = test_set
results_df = run_experiment(results_df, model, 5, get_zero_shot_prompt) 
results_df.to_csv(f"results/cleaned_dataset/{model.replace("/", "-")}_zero_shot_06052024.csv")

/var/folders/zw/vnb4lkhd14n9cx0d71_qh_z00000gn/T/ipykernel_25859/2997118475.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[[f"raw_output_{i}", f"model_confidence_classification_{i}", f"model_logprobs{i}"]] = dataset["final_statement"].apply(lambda x: query_together_model(client, x, prompt_fn, model))
/var/folders/zw/vnb4lkhd14n9cx0d71_qh_z00000gn/T/ipykernel_25859/2997118475.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[[f"raw_output_{i}", f"model_confidence_classification_{i}", f

In [ ]:
results_df.head()

In [ ]:
model_classification_col_names = [f"model_confidence_classification_{i}" for i in range(1, 6)]
print_accuracy_slope_bias_metrics(results_df, model_classification_col_names, plot=False, verbose=False)